In [1]:

import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import datasets, linear_model
import gc
%matplotlib inline
from IPython.display import display, HTML
from pprint import pprint

In [2]:
P = pd.read_csv("../../input/producto_tabla.csv")

In [3]:
#take 1 CSV, then split it to 3..
class FeatureEngineering:

    def __init__(self, ValidationStart, ValidationEnd, trainHdfPath, trainHdfFile, testHdfPath1, testHdfPath2, testHdfFile, 
                 testTypes, trainTypes, trainCsvPath, testCsvPath, maxLag=0):
        self.ValidationStart = ValidationStart
        self.ValidationEnd = ValidationEnd
        self.maxLag = maxLag
        self.trainHdfPath = trainHdfPath
        self.trainHdfFile = trainHdfFile
        self.testHdfPath1 = testHdfPath1
        self.testHdfPath2 = testHdfPath2
        self.testHdfFile = testHdfFile
        self.testTypes = testTypes
        self.trainTypes = trainTypes
        self.trainCsvPath = trainCsvPath
        self.testCsvPath = testCsvPath
        
    @staticmethod
    def __printDataFrameBasics__(data):
        display(data.head(2))
        #print data.dtypes
        gc.collect()
        print(data.info(memory_usage=True))
        
    @staticmethod    
    def changeIndexTypeToLowerMemory(data):
        ##########
        #This is very critical, i accept max number is 2^32. Also, if don't do that, memory gets so much higher..
        ##########
        #data.reset_index(inplace=True)
        #data.drop("index",axis=1, inplace=True)
        data.index = data.index.astype('uint32')
        gc.collect()
        
    def ReadHdf(self, trainOrTestOrBoth):
        '''Reads and holds Df in object memory'''            
        if trainOrTestOrBoth == 'train' or trainOrTestOrBoth=='both':
            self.train = pd.read_hdf(self.trainHdfPath,self.trainHdfFile)
            FeatureEngineering.changeIndexTypeToLowerMemory(self.train)
            FeatureEngineering.__printDataFrameBasics__(self.train)
            
        if trainOrTestOrBoth == 'test' or trainOrTestOrBoth=='both':
            self.test1 = pd.read_hdf(self.testHdfPath1,self.testHdfFile)
            self.test2 = pd.read_hdf(self.testHdfPath2,self.testHdfFile)
            FeatureEngineering.changeIndexTypeToLowerMemory(self.test1)
            FeatureEngineering.changeIndexTypeToLowerMemory(self.test2)
            FeatureEngineering.__printDataFrameBasics__(self.test1)
            FeatureEngineering.__printDataFrameBasics__(self.test2)
        
    def ReadCsv(self, trainOrTestOrBoth):
        '''Reads and holds Df in memory'''
        if trainOrTestOrBoth == 'train' or trainOrTestOrBoth == 'both':
            self.train = pd.read_csv(self.trainCsvPath, usecols=self.trainTypes.keys(), dtype=self.trainTypes)
            FeatureEngineering.changeIndexTypeToLowerMemory(self.train)
            FeatureEngineering.__printDataFrameBasics__(self.train)
        if trainOrTestOrBoth == 'test' or trainOrTestOrBoth=='both':
            tempTest = pd.read_csv(self.testCsvPath, usecols=self.testTypes.keys(), dtype=self.testTypes)
            self.test1 = tempTest.loc[tempTest.Semana == self.ValidationStart]
            self.test2 = tempTest.loc[tempTest.Semana == self.ValidationEnd]
            del tempTest
            FeatureEngineering.changeIndexTypeToLowerMemory(self.test1)
            FeatureEngineering.changeIndexTypeToLowerMemory(self.test2)
            FeatureEngineering.__printDataFrameBasics__(self.test1)
            FeatureEngineering.__printDataFrameBasics__(self.test2)
            
    @staticmethod
    def ConvertCsvToHdf(csvPath, HdfPath, HdfName, ColumnTypeDict ):
        tempDf = pd.read_csv(csvPath, usecols=ColumnTypeDict.keys(), dtype=ColumnTypeDict,index=False)
        tempDf.to_hdf(HdfPath, HdfName, format='t')
        del tempDf
        gc.collect()
        print "ConvertCsvToHdf is done.."

    def Preprocess(self, trainOrTestOrBoth, columnFunctionTypeList):
        '''columnFunctionTypeList = [ ['C1',Func1,Type], ['C2',Func2,Type],..    ]'''
        for column, func, localType in columnFunctionTypeList:
            if trainOrTestOrBoth == 'train' or trainOrTestOrBoth =='both':
                self.train.loc[:,column] =  self.train[column].apply(func).astype(localType)
            if trainOrTestOrBoth == 'test' or trainOrTestOrBoth == 'both':
                self.test1.loc[:,column] =  self.test1[column].apply(func).astype(localType)
                self.test2.loc[:,column] =  self.test2[column].apply(func).astype(localType)
        gc.collect()
        
    def SaveDataFrameToHdf(self,trainOrTestOrBoth):
        if trainOrTestOrBoth == 'train' or trainOrTestOrBoth=='both':
            self.train.to_hdf(self.trainHdfPath, self.trainHdfFile, format='t', index="False")
        if trainOrTestOrBoth == 'test' or trainOrTestOrBoth=='both':
            self.test1.to_hdf(self.testHdfPath1, self.testHdfFile, format='t', index="False")
            self.test2.to_hdf(self.testHdfPath2, self.testHdfFile, format='t', index="False")
        
    def AddDemandaGeneralMean(self): 
        self.train.loc[:,"DemandaGeneralMean"] = self.train["Demanda_uni_equil"].loc[
            self.train.loc[:,'Semana'] < 10].mean().astype("float32")
        #self.train.loc[:,"DemandaGeneralMean"] = DemandaMeanWithoutLag['Demanda_uni_equil'].mean()
        #self.train.loc[:,"DemandaGeneralMean"] = self.train.loc[:,"DemandaGeneralMean"].astype('float32')
        #display(self.train)
        #del DemandaMeanWithoutLag
        gc.collect()
        
    '''ConfigElements(0,[ ("A",["Semana","Agencia_ID"],["count","count"]),'''
    def AddConfigurableFeaturesToTrain(self, config):
        if config.lag > self.maxLag:
            self.maxLag = config.lag
        
        tempData = self.train.loc[self.train.loc[:,'Semana'] <= self.ValidationEnd - config.lag]
        #display(tempData)
        if(config.lag != 0):
            tempData.loc[:,'Semana'] = tempData['Semana'].apply(lambda x:x + config.lag)
        #display(tempData)
        
        #Means iterative.. eliminate as long as np.nan exists..If there is already one, don't create, use the existing
        if config.targetVariable != "" and  config.targetVariable not in self.train.columns:
            self.train.loc[:,config.targetVariable] = np.nan
            self.test1.loc[:,config.targetVariable] = np.nan
            
            if config.lag != 1:
                self.test2.loc[:,config.targetVariable] = np.nan
        
        for name,groups,aggregate in config.nameAndGroups:
            if name not in self.train.columns:
                print "{} is not in columns..".format(name)            
                
                groupedDataframe = tempData[groups+['Demanda_uni_equil']].groupby(groups).agg(aggregate[0])
                #groupedDataframe.columns = groupedDataframe.columns.droplevel(0)
                groupedDataframe.columns = [name]
                
                #This is means of the counts of the semana-columns tuples!..!!!
                #If no lag and mean, mean of the columns without semana!!..
                #If there is lag and count, count of the columns x weeks before
                #If there is lag and mean, mean of the columns x weeks before
                #if(config.lag == 0 and aggregate == "count"):
                if(len(aggregate)>1):
                    groupedDataframe.reset_index(inplace=True)
                    groupedDataframe.drop("Semana",axis=1, inplace=True)
                    groups = groups[1:]
                    groupedDataframe = groupedDataframe.groupby(groups).agg(aggregate[1])
                    groupedDataframe.columns = [name]
                    gc.collect()
                
                display(groupedDataframe)
                self.train = self.train.merge( groupedDataframe, left_on=groups,
                    right_index=True, how='left', sort=False,copy=False)
                self.test1 = self.test1.merge( groupedDataframe, left_on=groups,
                    right_index=True, how='left', sort=False,copy=False)
                if config.lag != 1:
                    self.test2 = self.test2.merge( groupedDataframe, left_on=groups,
                        right_index=True, how='left', sort=False,copy=False)
                
                del groupedDataframe
                gc.collect()
            else:
                print "{} is in columns..".format(name)
            
            display(self.train)
            display(self.test1)
            display(self.test2)
            
            #Means iterative..!!!!!
            if config.targetVariable != "":
                self.train.loc[pd.isnull(self.train[config.targetVariable]), 
                    config.targetVariable] = self.train.loc[pd.isnull(self.train[config.targetVariable]), name]
                self.test1.loc[pd.isnull(self.test1[config.targetVariable]), 
                    config.targetVariable] = self.test1.loc[pd.isnull(self.test1[config.targetVariable]), name]
                if config.lag != 1:
                    self.test2.loc[pd.isnull(self.test2[config.targetVariable]), 
                        config.targetVariable] = self.test2.loc[pd.isnull(self.test2[config.targetVariable]), name]
                    
                count = self.test1.loc[:,config.targetVariable].isnull().sum()
                print "Count of missing numbers after {} in validation part 1 in column {} is {}".format(name, 
                    config.targetVariable,str(count))
                if config.lag != 1:
                    count = self.test2.loc[:,config.targetVariable].isnull().sum()
                    print "Count of missing numbers after {} in validation part 2 in column {} is {}".format(name, 
                        config.targetVariable,str(count))
                
                
                #display(self.train)
                #If column is already in Dataframe and we want to fill target variable, this deletes columns!!!
                if(config.deleteColumns):
                    self.train.drop(name, axis=1, inplace=True)
                    self.test1.drop(name, axis=1, inplace=True)
                    if config.lag != 1:
                        self.test2.drop(name, axis=1, inplace=True)
                gc.collect()
                #Only in tesst
                #if count == 0:
                 #   break
        del tempData
        display(self.train)   
        display(self.test1)   
        display(self.test2)
        gc.collect()
        return 
    
    def DeleteLaggedWeeksFromTrain(self,trainOrTestOrBoth):
        self.train = self.train.loc[self.train.loc[:,'Semana']>= 3 + self.maxLag]
        gc.collect()
        display(self.train.head(2))
        
    def ReadFirstNRowsOfACsv(self, nrows, trainOrTestOrBoth) :
        if trainOrTestOrBoth == 'train' or trainOrTestOrBoth=='both':
            self.train = pd.read_csv(self.trainCsvPath, usecols=self.trainTypes.keys(), dtype=self.trainTypes, nrows = nrows)
            FeatureEngineering.changeIndexTypeToLowerMemory(self.train)
            FeatureEngineering.__printDataFrameBasics__(self.train)
        if trainOrTestOrBoth == 'test' or trainOrTestOrBoth=='both':
            tempTest = pd.read_csv(self.testCsvPath, usecols=self.testTypes.keys(), dtype=self.testTypes, nrows = nrows*2)
            self.test1 = tempTest.loc[tempTest.Semana == self.ValidationStart]
            self.test2 = tempTest.loc[tempTest.Semana == self.ValidationEnd]
            del tempTest
            FeatureEngineering.changeIndexTypeToLowerMemory(self.test1)
            FeatureEngineering.changeIndexTypeToLowerMemory(self.test2)
            FeatureEngineering.__printDataFrameBasics__(self.test1)
            FeatureEngineering.__printDataFrameBasics__(self.test2)
    
    #Use when concatanating train and validation before predict test for example..
    def AppendTestToTrain(self,deleteTest = True):
        self.train = self.train.append(self.test1,ignore_index=True)
        gc.collect()
        if(deleteTest):
            del self.test1
            gc.collect()
        try:
            self.train = self.train.append(self.test2,ignore_index=True)
            gc.collect()
            if(deleteTest):
                del self.test2
                gc.collect()
        except:
            pass
    #Split train data to train and test1 and test2 (validation)
    def SplitTrainToTestUsingValidationStart(self):
        boolCondition = self.train.Semana == self.ValidationStart
        self.test1 = self.train.loc[boolCondition]
        self.train.drop((self.train.loc[boolCondition].index), axis=0,inplace=True)
        gc.collect()
        
        boolCondition = self.train.Semana == self.ValidationEnd
        self.test2 = self.train.loc[boolCondition]
        self.train.drop((self.train.loc[boolCondition].index), axis=0,inplace=True)
        
        del boolCondition
        gc.collect()


In [4]:
parameterDict =       {"ValidationStart":10, 
 "ValidationEnd":11,
   "maxLag":2,
    "trainHdfPath":'../../input/train.h5',
    "trainHdfFile":"train",
    "testHdfPath1":"../../input/test1.h5",
    "testHdfPath2":"../../input/test2.h5",
    "testHdfFile":"test", 
    "trainTypes" : {'Semana':np.uint8, 'Agencia_ID':np.uint16, 'Canal_ID':np.uint8,'Ruta_SAK':np.uint16, 
        'Cliente_ID':np.uint32, 'Producto_ID':np.uint16,'Venta_uni_hoy':np.uint16, 'Venta_hoy':np.float32,
                    'Dev_uni_proxima': np.uint32, 'Dev_proxima':np.float32,'Demanda_uni_equil':np.uint32}, 
    "testTypes" : {'id':np.uint32,'Semana':np.uint8, 'Agencia_ID':np.uint16, 'Canal_ID':np.uint8,'Ruta_SAK':np.uint16,
        'Cliente_ID':np.uint32, 'Producto_ID':np.uint16},
    "trainCsvPath":'../../input/train.csv'   ,
    "testCsvPath":'../../input/test.csv'}

FE = FeatureEngineering(**parameterDict)
print FE.__dict__

{'trainCsvPath': '../../input/train.csv', 'maxLag': 2, 'testTypes': {'Cliente_ID': <type 'numpy.uint32'>, 'Ruta_SAK': <type 'numpy.uint16'>, 'Canal_ID': <type 'numpy.uint8'>, 'Producto_ID': <type 'numpy.uint16'>, 'Agencia_ID': <type 'numpy.uint16'>, 'Semana': <type 'numpy.uint8'>, 'id': <type 'numpy.uint32'>}, 'testHdfFile': 'test', 'trainTypes': {'Dev_proxima': <type 'numpy.float32'>, 'Venta_uni_hoy': <type 'numpy.uint16'>, 'Cliente_ID': <type 'numpy.uint32'>, 'Demanda_uni_equil': <type 'numpy.uint32'>, 'Ruta_SAK': <type 'numpy.uint16'>, 'Canal_ID': <type 'numpy.uint8'>, 'Venta_hoy': <type 'numpy.float32'>, 'Producto_ID': <type 'numpy.uint16'>, 'Agencia_ID': <type 'numpy.uint16'>, 'Dev_uni_proxima': <type 'numpy.uint32'>, 'Semana': <type 'numpy.uint8'>}, 'testHdfPath1': '../../input/test1.h5', 'ValidationEnd': 11, 'testHdfPath2': '../../input/test2.h5', 'testCsvPath': '../../input/test.csv', 'ValidationStart': 10, 'trainHdfFile': 'train', 'trainHdfPath': '../../input/train.h5'}


In [5]:
FE.ReadHdf('train')

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,Demanda_uni_equil
0,3,1110,7,3301,15766,1212,3,25.139999,0,0,1.386294
1,3,1110,7,3301,15766,1216,4,33.520000,0,0,1.609438


<class 'pandas.core.frame.DataFrame'>
Int64Index: 74180464 entries, 0 to 74180463
Data columns (total 11 columns):
Semana               uint8
Agencia_ID           uint16
Canal_ID             uint8
Ruta_SAK             uint16
Cliente_ID           uint32
Producto_ID          uint16
Venta_uni_hoy        uint16
Venta_hoy            float32
Dev_uni_proxima      uint32
Dev_proxima          float32
Demanda_uni_equil    float32
dtypes: float32(3), uint16(4), uint32(2), uint8(2)
memory usage: 2.3 GB
None


In [6]:
#FE.train.loc[:,"Demanda_uni_equil"] = np.expm1(np.array(FE.train["Demanda_uni_equil"])).astype('uint32')

In [7]:
FE.train.loc[:,"Demanda_uni_equil"] = np.expm1(FE.train["Demanda_uni_equil"].values).astype('uint32')

In [8]:
FE.train.dtypes

Semana                 uint8
Agencia_ID            uint16
Canal_ID               uint8
Ruta_SAK              uint16
Cliente_ID            uint32
Producto_ID           uint16
Venta_uni_hoy         uint16
Venta_hoy            float32
Dev_uni_proxima       uint32
Dev_proxima          float32
Demanda_uni_equil     uint32
dtype: object

In [9]:
gc.collect()

14

In [10]:
P.sort_values(by=["NombreProducto"])[50:80]

AttributeError: 'DataFrame' object has no attribute 'sort_values'

## Venta and Demanda Summary

In [169]:
WholeWheat = FE.train.loc[FE.train.Producto_ID.isin([42436,42437,43377,46299,42351])]

In [170]:
gc.collect()

97

In [171]:
WholeWheat.sort_values(by=["Semana","Cliente_ID"],inplace=True)

C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [172]:
WholeWheatGroup = WholeWheat.groupby(["Semana"])["Venta_uni_hoy","Demanda_uni_equil"].agg(["sum","mean","std","count"])
WholeWheatGroup

Venta_uni_hoy                             Demanda_uni_equil             \
                 sum       mean        std count               sum       mean   
Semana                                                                          
3              35567  46.432115  55.470819   766             34793  45.421671   
4              36914  47.265045  69.224126   781             36296  46.473752   
5              31460  43.633842  46.327538   721             30840  42.773925   
6              33162  42.515385  67.857122   780             32768  42.010256   
7              33479  40.336145  52.712059   830             32695  39.391566   
8              32411  35.538377  41.097521   912             31913  34.992325   
9              31754  36.752315  36.649462   864             31335  36.267361   

                         
              std count  
Semana                   
3       54.620602   766  
4       68.806350   781  
5       46.070630   721  
6       67.932742   780  
7       52.582014   830  
8       40.956684   912  
9       36.550307   864

## Weekly summary

In [173]:
WholeWheatGroup = WholeWheat.groupby(["Semana","Producto_ID"])["Venta_uni_hoy","Demanda_uni_equil"].agg(
    ["sum","mean","std","count"])
WholeWheatGroup

Venta_uni_hoy                              \
                             sum       mean        std count   
Semana Producto_ID                                             
3      42436                1084   9.678571  11.709097   112   
       42437               22831  67.949405  67.297871   336   
       43377               11443  37.765677  39.638113   303   
       46299                 209  13.933333  10.436657    15   
4      42436                1035   9.495413  14.870816   109   
       42437               24331  70.524638  91.114706   345   
       43377               11222  35.625397  38.455442   315   
       46299                 326  27.166667  19.058920    12   
5      42436                1012   8.955752   9.787373   113   
       42437               19947  65.615132  54.907963   304   
       43377               10170  34.948454  32.448486   291   
       46299                 331  25.461538  23.613610    13   
6      42436                1411  10.937984  12.418328   129   
       42437               20191  63.096875  92.990174   320   
       43377               11072  35.261146  40.684816   314   
       46299                 488  28.705882  20.532793    17   
7      42436                1284   9.441176  11.530806   136   
       42437               20735  63.024316  69.708010   329   
       43377               10871  31.693878  31.828343   343   
       46299                 589  26.772727  27.916426    22   
8      42436                 988   7.718750   8.190582   128   
       42437               20413  48.143868  50.111371   424   
       43377               10489  30.669591  29.063873   342   
       46299                 521  28.944444  16.408291    18   
9      42436                1202   9.693548  15.694325   124   
       42437               20014  50.160401  40.370761   399   
       43377                9939  30.675926  30.249784   324   
       46299                 599  35.235294  25.158322    17   

                   Demanda_uni_equil                              
                                 sum       mean        std count  
Semana Producto_ID                                                
3      42436                    1083   9.669643  11.716154   112  
       42437                   22160  65.952381  66.300115   336  
       43377                   11342  37.432343  39.614967   303  
       46299                     208  13.866667  10.370746    15  
4      42436                    1034   9.486239  14.870811   109  
       42437                   23823  69.052174  90.706194   345  
       43377                   11113  35.279365  38.508971   315  
       46299                     326  27.166667  19.058920    12  
5      42436                    1010   8.938053   9.802772   113  
       42437                   19534  64.256579  55.048073   304  
       43377                    9965  34.243986  32.047039   291  
       46299                     331  25.461538  23.613610    13  
6      42436                    1406  10.899225  12.444781   129  
       42437                   19926  62.268750  93.290474   320  
       43377                   10960  34.904459  40.632819   314  
       46299                     476  28.000000  20.472543    17  
7      42436                    1274   9.367647  11.546241   136  
       42437                   20154  61.258359  69.889352   329  
       43377                   10678  31.131195  31.934376   343  
       46299                     589  26.772727  27.916426    22  
8      42436                     987   7.710938   8.197515   128  
       42437                   20119  47.450472  50.294945   424  
       43377                   10289  30.084795  28.401217   342  
       46299                     518  28.777778  16.347533    18  
9      42436                    1202   9.693548  15.694325   124  
       42437                   19622  49.177945  40.500709   399  
       43377                    9913  30.595679  30.231103   324  
       46299            

## Total Client together vs all clients they are sold

In [138]:
WholeWheat["Cliente_ID"].count()

3899

In [139]:
len(WholeWheat["Cliente_ID"].unique())

18

In [140]:
filtered = WholeWheat.loc[:,["Cliente_ID"]+['Demanda_uni_equil']].groupby(["Cliente_ID"]).count()
filtered = filtered[filtered.loc[:,'Demanda_uni_equil']>1]
filtered.columns = ['count']
filtered.merge(WholeWheat,left_index = True, right_on= ["Cliente_ID"], how ="left").sort_values(by=["Semana","Cliente_ID"])

,count,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,Demanda_uni_equil
10793331,9,3,4081,5,3002,652796,40640,0,0.000000,3,30.0,0
7000819,21,3,1915,5,3004,652850,40639,8,66.639999,14,140.0,0
7000820,21,3,1915,5,3004,652850,40640,11,98.309998,0,0.0,11
6511474,63,3,1654,5,3005,653037,40639,148,1309.660034,0,0.0,148
6511475,63,3,1654,5,3005,653037,40640,188,1669.579956,0,0.0,188
6511476,63,3,1654,5,3005,653037,40641,237,2041.010010,0,0.0,237
6511516,63,3,1654,5,3006,653037,40639,88,808.190002,1,10.0,87
6511517,63,3,1654,5,3006,653037,40640,86,799.880005,0,0.0,86
6511518,63,3,1654,5,3006,653037,40641,91,848.210022,0,0.0,91
6511561,63,3,1654,5,3007,653037,40639,77,723.239990,0,0.0,77


In [132]:
WholeWheat.loc[WholeWheat.Cliente_ID == 653378]

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,Demanda_uni_equil
2219635,3,1229,5,3001,653378,35631,86,478.160004,0,0.000000,86
2219734,3,1229,5,3003,653378,35632,137,761.719971,2,11.120000,135
2219834,3,1229,5,3005,653378,35632,58,322.480011,0,0.000000,58
2651231,3,1250,5,3001,653378,35632,12,66.720001,0,0.000000,12
2651358,3,1250,5,3004,653378,35631,48,266.880005,0,0.000000,48
2651492,3,1250,5,3007,653378,35632,92,511.519989,0,0.000000,92
2651536,3,1250,5,3008,653378,35631,36,200.160004,0,0.000000,36
2651582,3,1250,5,3009,653378,35631,120,667.200012,0,0.000000,120
2651583,3,1250,5,3009,653378,35632,120,667.200012,0,0.000000,120
2651636,3,1250,5,3010,653378,35631,113,628.280029,0,0.000000,113
